In [ ]:
import mysql.connector
import tkinter as tk
from tkinter import ttk, messagebox
import random
from faker import Faker
from datetime import datetime, timedelta

# Initialize Faker for generating random data
fake = Faker()

# MySQL Database Connection Details
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "root",
    "database": "crop_management"
}

# List of sample crop names
crop_names = ["Wheat", "Rice", "Corn", "Soybean", "Barley", "Sugarcane", "Cotton", "Potato", "Tomato", "Lettuce"]

# List of possible growth stages
growth_stages = ["Seedling", "Vegetative", "Flowering", "Fruiting", "Maturity"]

# List of sample pest control measures
pest_control_measures_list = [
    "Use of organic pesticides",
    "Crop rotation",
    "Neem oil application",
    "Biological pest control",
    "Chemical pesticides",
    "Regular field monitoring",
]

# Database Connection Function
def connect_db():
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        return conn
    except mysql.connector.Error as e:
        messagebox.showerror("Database Error", f"Error connecting to database: {e}")
        return None

# Function to Insert Manual Crop Record
def insert_manual_record():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        crop_name = crop_name_entry.get()
        planting_date = planting_date_entry.get()
        harvest_date = harvest_date_entry.get()
        growth_stage = growth_stage_entry.get()
        pest_control = pest_control_entry.get()
        yield_prediction = yield_entry.get()

        if not crop_name or not planting_date or not harvest_date or not growth_stage or not pest_control or not yield_prediction:
            messagebox.showwarning("Input Error", "All fields must be filled!")
            return

        try:
            cursor.execute("""
                INSERT INTO crops (crop_name, planting_date, harvest_date, growth_stage, pest_control_measures, yield_prediction)
                VALUES (%s, %s, %s, %s, %s, %s)
            """, (crop_name, planting_date, harvest_date, growth_stage, pest_control, yield_prediction))
            conn.commit()
            messagebox.showinfo("Success", "Crop record inserted successfully!")
            conn.close()
            display_records()
        except mysql.connector.Error as e:
            messagebox.showerror("Database Error", f"Error inserting record: {e}")

# Function to Generate Random Data for Bulk Insert
def generate_data():
    crop_name = random.choice(crop_names)
    planting_date = fake.date_between(start_date="-2y", end_date="today")  # Planting in last 2 years
    harvest_date = planting_date + timedelta(days=random.randint(60, 180))  # Harvest after 2-6 months
    growth_stage = random.choice(growth_stages)
    pest_control = random.choice(pest_control_measures_list)
    yield_prediction = random.randint(500, 5000)  # Yield in kg
    return (crop_name, planting_date, harvest_date, growth_stage, pest_control, yield_prediction)

# Function to Insert 100,000 Random Records
def insert_bulk_records():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        batch_size = 100000
        total_records = 2000000

        for i in range(0, total_records, batch_size):
            data_batch = [generate_data() for _ in range(batch_size)]
            cursor.executemany("""
                INSERT INTO crops (crop_name, planting_date, harvest_date, growth_stage, pest_control_measures, yield_prediction)
                VALUES (%s, %s, %s, %s, %s, %s)
            """, data_batch)
            conn.commit()
            progress_label.config(text=f"{i + batch_size} records inserted...")

        messagebox.showinfo("Success", "100,000 records inserted successfully!")
        conn.close()
        display_records()

# Function to Display Records
def display_records():
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM crops ORDER BY id DESC LIMIT 20")  # Show last 20 records
        rows = cursor.fetchall()
        conn.close()

        for row in tree.get_children():
            tree.delete(row)

        for row in rows:
            tree.insert("", "end", values=row)

# GUI Setup
root = tk.Tk()
root.title("Crop Management System")
root.geometry("900x600")
root.configure(bg="#f0f0f0")

# Custom Fonts and Colors
font_title = ('Helvetica', 16, 'bold')
font_label = ('Arial', 12)
font_entry = ('Arial', 12)
font_button = ('Arial', 12, 'bold')
font_progress = ('Courier', 10, 'italic')

# Title Label
title_label = tk.Label(root, text="Crop Management System", font=('Helvetica', 20, 'bold'), bg="#4CAF50", fg="white", pady=10)
title_label.grid(row=0, column=0, columnspan=2, sticky="ew")

# Input Fields with Padding and Styling
tk.Label(root, text="Crop Name", font=font_label, bg="#f0f0f0").grid(row=1, column=0, padx=10, pady=5)
crop_name_entry = tk.Entry(root, font=font_entry)
crop_name_entry.grid(row=1, column=1, padx=10, pady=5)

tk.Label(root, text="Planting Date (YYYY-MM-DD)", font=font_label, bg="#f0f0f0").grid(row=2, column=0, padx=10, pady=5)
planting_date_entry = tk.Entry(root, font=font_entry)
planting_date_entry.grid(row=2, column=1, padx=10, pady=5)

tk.Label(root, text="Harvest Date (YYYY-MM-DD)", font=font_label, bg="#f0f0f0").grid(row=3, column=0, padx=10, pady=5)
harvest_date_entry = tk.Entry(root, font=font_entry)
harvest_date_entry.grid(row=3, column=1, padx=10, pady=5)

tk.Label(root, text="Growth Stage", font=font_label, bg="#f0f0f0").grid(row=4, column=0, padx=10, pady=5)
growth_stage_entry = tk.Entry(root, font=font_entry)
growth_stage_entry.grid(row=4, column=1, padx=10, pady=5)

tk.Label(root, text="Pest Control Measures", font=font_label, bg="#f0f0f0").grid(row=5, column=0, padx=10, pady=5)
pest_control_entry = tk.Entry(root, font=font_entry)
pest_control_entry.grid(row=5, column=1, padx=10, pady=5)

tk.Label(root, text="Yield Prediction (kg)", font=font_label, bg="#f0f0f0").grid(row=6, column=0, padx=10, pady=5)
yield_entry = tk.Entry(root, font=font_entry)
yield_entry.grid(row=6, column=1, padx=10, pady=5)

# Buttons with Styling
insert_button = tk.Button(root, text="Insert Record", font=font_button, bg="#4CAF50", fg="white", command=insert_manual_record)
insert_button.grid(row=7, column=0, columnspan=2, pady=10)

bulk_insert_button = tk.Button(root, text="Insert 100,000 Random Records", font=font_button, bg="#008CBA", fg="white", command=insert_bulk_records)
bulk_insert_button.grid(row=8, column=0, columnspan=2, pady=10)

progress_label = tk.Label(root, text="", font=font_progress, bg="#f0f0f0")
progress_label.grid(row=9, column=0, columnspan=2)

# Table to Display Records with Enhanced Style
columns = ("ID", "Crop Name", "Planting Date", "Harvest Date", "Growth Stage", "Pest Control", "Yield Prediction")
tree = ttk.Treeview(root, columns=columns, show="headings", height=10)
tree.grid(row=10, column=0, columnspan=2, padx=10, pady=10)

# Style for Treeview
tree.heading("ID", text="ID")
tree.heading("Crop Name", text="Crop Name")
tree.heading("Planting Date", text="Planting Date")
tree.heading("Harvest Date", text="Harvest Date")
tree.heading("Growth Stage", text="Growth Stage")
tree.heading("Pest Control", text="Pest Control")
tree.heading("Yield Prediction", text="Yield Prediction (kg)")

tree.column("ID", width=50, anchor="center")
tree.column("Crop Name", width=150, anchor="center")
tree.column("Planting Date", width=120, anchor="center")
tree.column("Harvest Date", width=120, anchor="center")
tree.column("Growth Stage", width=100, anchor="center")
tree.column("Pest Control", width=180, anchor="center")
tree.column("Yield Prediction", width=120, anchor="center")

# Load initial records
display_records()

# Run the GUI
root.mainloop()
